이제 데이터를 처리하고난 후에 수행할 작업들을 알아보자.

# Long-term memory

스트림 데이터를 비스트리밍에 맞게 설계된 저장소에 저장하려면(AWS S3, HDFS, HBase, RDBMS) 몇 가지 구현할 수 있는 방법이 있다. 이때 인메모리 방법을 쓸 수도 있지만, 장기 스토리지에 저장하는 방식을 보자.  

### 직접 저장(Direct Writing)

스트림으로 들어오는 데이터를 지속적으로 저장할 수 있다. 또는 특정 배치 크기에 도달하거나 일정 시간이 경과할 때까지 분석 단계에서 데이터를 가지고 있다가 저장하는 방식이 있다. 이런 방식은 데이터의 볼륨과 속가 증가함에 따라 이슈가 발생할 가능성이 크다. 

### 간접 저장(Indirect Writing)

스트림 데이터가 유입되는 즉시 메시지를 저장하고, 배치로 기다린 후에 스토리지로 이동한다. 다만 메시지 큐 단계에 저장함으로써 스트리밍 분석 단계와 장기 스토리지의 커플링을 분리한다. 따라서 Batch Loader를 구현해서 이녀석이 메시지 큐 단계에서 배치 메시지를 읽고 장기 스토리지에 저장하도록 만들어야 한다.  

Goblin, Secor 등으로 배치 저장을 수행할 수 있다.  

# 인메모리 저장

이벤트가 발생했을 때 가능한 빨리 실시간으로 데이터를 처리하려면 인메모리 저장소가 반드시 필요하다. 원래는 Caching에서만 메모리에 많은 데이터를 보관했는데, 지금은 스트리밍 시스템의 전체 데이터를 메모리에 보관하는 것도 불가능한 일이 아니다.  

SSD도 빠르지 않냐고 물어본다면, SSD는 10만 나노 초가 소요될 때 메인 메모리에 접근하는 데에는 100나노 초가 걸리고 L1 캐시에 접근하는 데에는 0.5나노 초밖에 걸리지 않는다. 아무리 데이터를 잘게 나눠도 SSD보다 DRAM에 접근하는 게 훨씬 빠르다.

### 인메모리/플래시 메모리 기반 DB

기업들에서는 인메모리나 플래시 메모리를 저장소로 가지는 내장형 DB가 많이 사용된다. 이런 DB는 전통적인 DB가 제공하는 관리 기능을 지원하지는 않고, 분산 스트리밍 시스템에도 적합하지 않다. SQLite, RocksDB, LevelDB, LMDB, Perst 등이 있다.

### Caching system

캐싱 시스템은 메모리에 데이터를 저장할 수 잇게 설계되었다. API는 키-값 기반으로 구현되어 있다. 일반적인 캐싱 전략은 아래와 같다.    

- **Read-through** : 캐시에 없는 항목을 요청받았을 때 영구 저장소에서 데이터를 읽어온다.  

- **Refresh-ahead** : 캐시가 만료/삭제되기 전에 가장 최근에 접근했던 데이터로 새로 바꾼다. 저장소에 데이터를 검색해야하는 Read-through보다 성능 저하가 덜하다.  

- **Write-through** : 캐싱 시스템이 업데이트된 데이터를 Backing store에 저장한다. 따라서 변경사항을 캐시에 로드하는 Out-of-band 처리를 없앤다

- **Write-around** : 캐시가 영구 저장소를 업데이트해서 Out-of-band 현상을 발생시킨다.  

- **Write-back** : 캐싱 시스템이 영구 저장소에 새로운 데이터를 쓴다.  

캐싱 시스템에 관심있다면 Memcached, EHCache, Hazelcast, Redis 등을 공부해보자

### 인메모레 DB/인메모리 data grid

인메모리 DB는 IMDB(In Memory DataBase) 또는 IMDG(In Memory Data Grids)라고 불린다. 캐싱 시스템과 다르게 데이터 영속성 보장을 위해 디스크를 사용하도록 설계되었다. DRAM이 아닌 저장 공간을 사용하는데, 메모리에 우선 저장하고 디스크에 저장한다는 특징이 있다. 전통적인 DB(Oracle 등), Apache Cassandra 등 현대 NoSQL DB는 디스크에 우선 데이터를 저장하고 메모리를 보조로 사용하도록 설계되어 있다.  

전통적인 DB가 인메모리 저장 기능을 추가할 때 동일 설계 원칙이 적용되고, 메모리는 디스크 대신 사용되는 또다른 I/O 장치처럼 운영된다. 메모리를 사용하지 않는게 아니라 성능상 이유로 메모리를 캐시로 사용하는 것이다.